# APIs

**Prerequisites:** You need the `requests` package for this lesson.  You should be able to install it with `conda`.

## Reading Existing Code (Google Books Query)
 - Uses the Python [`requests` library](https://2.python-requests.org/en/master/)
 - Uses the [Google Books API](https://developers.google.com/books/docs/v1/using#WorkingVolumes)
 - Searches for author "Jake VanderPlas" (author of my go-to Python data science book)
 - Prints out a list of book titles that match this query

In [6]:
import requests

response = requests.get("https://www.googleapis.com/books/v1/volumes?q=inauthor:Stephen+King")

if response.status_code == 200:
    response_dict = response.json()

    books = response_dict["items"]

    for book_dict in books:
        info_dict = book_dict["volumeInfo"]
        print(info_dict["title"])
else:
    print("Error: unable to retrieve books.  Server responded with status code", response.status_code)

Stephen King, American Master
It
On Writing
Night Shift
The Outsider
Danse Macabre
Carrie
Cell
The Mist
Stephen King Goes to the Movies


## Exercise

Use the PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [7]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

Read the documentation [here](https://pokeapi.co/docs/v2.html) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
 - ID
 - Name
 - Base experience
 - Weight
 - Height
 - Types
 - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```

In [13]:
# helper functions for types and abilities

def type_names(info):        
    types = info["types"]
    return [type_['type']['name'] for type_ in types]


def ability_names(info):
    abilities = info["abilities"]
    return [ability['ability']['name'] for ability in abilities]


In [14]:
def get_pokedata(url):
    
    """
    url is a string of the URL that will get you the relevant info from the API
    for a single pokemon. 
    Example: "https://pokeapi.co/api/v2/pokemon/1/"
    
    this function should return the dictionary for ONE pokemon
    
    """
    response = requests.get(url)
    
    if response.status_code == 200:
        info = response.json()
    
        # list of keys with values that don't need editing
        keys = ['id', 'name', 'base_experience', 'weight', 'height'] 
        # dictionary comprehension to extract those keys
        data = {k: info[k] for k in keys} 

        # using the two helper functions to add types and abilities
        data['types'] = type_names(info)
        data['abilities'] = ability_names(info)

        return data
    else:
        return {}

In [15]:
get_pokedata("https://pokeapi.co/api/v2/pokemon/1/")

{'id': 1,
 'name': 'bulbasaur',
 'base_experience': 64,
 'weight': 69,
 'height': 7,
 'types': ['poison', 'grass'],
 'abilities': ['chlorophyll', 'overgrow']}

In [16]:
get_pokedata("https://pokeapi.co/api/v2/pokemon/151/")

{'id': 151,
 'name': 'mew',
 'base_experience': 270,
 'weight': 40,
 'height': 4,
 'types': ['psychic'],
 'abilities': ['synchronize']}

### Pagination

Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination)

Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```

In [22]:
import time
def get_151_pokemon():
    url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
    results = requests.get(url).json()['results']
    
    urls = [result["url"] for result in results]
    
    first_99_urls = urls[:99]
    remaining_urls = urls[99:]
    
    first_99_pokemon = get_pokemon(first_99_urls)
    time.sleep(60)
    remaining_pokemon = get_pokemon(remaining_urls)
    
    first_99_pokemon.extend(remaining_pokemon)
    
    return first_99_pokemon

def get_pokemon(urls):
    pokemon = []
    for url in urls:
        data = get_pokedata(url)
        pokemon.append(data)
    return pokemon
        
    

In [23]:
pokemon = get_151_pokemon()

In [27]:
pokemon[0]

{'id': 1,
 'name': 'bulbasaur',
 'base_experience': 64,
 'weight': 69,
 'height': 7,
 'types': ['poison', 'grass'],
 'abilities': ['chlorophyll', 'overgrow']}

In [29]:
pokemon[150]

{'id': 151,
 'name': 'mew',
 'base_experience': 270,
 'weight': 40,
 'height': 4,
 'types': ['psychic'],
 'abilities': ['synchronize']}

## Bonus Exercise
1. Query the [NASA ISS API](http://api.open-notify.org/) and print out the current latitude and longitude of the ISS
2. Using iPyLeaflet, make a [Map](https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html) with a [Marker](https://ipyleaflet.readthedocs.io/en/latest/api_reference/marker.html) at the current location of the ISS

In [24]:
from ipyleaflet import Map, Marker

In [25]:
response = requests.get("http://api.open-notify.org/iss-now.json")
response_dict = response.json()

iss_position = response_dict["iss_position"]

lat = iss_position["latitude"]
long = iss_position["longitude"]

location=(float(lat), float(long))

In [26]:
map_ = Map(
    center=location,
    zoom=3)
marker = Marker(location=location)
map_.add_layer(marker)
map_

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …